In [ ]:
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from skimage import io, transform, morphology, filters, segmentation, measure, color
from scipy import ndimage as nd
from tqdm import tqdm, notebook
from numba import jit
from collections import Counter
import cv2
  
def display_image(img, title='', size=None):
  #plt.subplot(1,2,1)
  plt.gray()
  h = plt.imshow(img, interpolation='none')
  if size:
    dpi = h.figure.get_dpi()/size
    h.figure.set_figwidth(img.shape[1] / dpi)
    h.figure.set_figheight(img.shape[0] / dpi)
    h.figure.canvas.resize(img.shape[1] + 1, img.shape[0] + 1)
    h.axes.set_position([0, 0, 1, 1])
    h.axes.set_xlim(-1, img.shape[1])
    h.axes.set_ylim(img.shape[0], -1)
  plt.grid(False)
  plt.title(title)  
  plt.show()

def display_label(img, title='', size=None):
  h, ax = plt.subplots()
  #ax.imshow(img, cmap=plt.cm.nipy_spectral, interpolation='nearest')
  cmap = colors.ListedColormap (np.random.rand (256,3))
  ax.imshow(img, cmap=cmap, interpolation='nearest')
  if size:
    dpi = ax.figure.get_dpi()/size
    ax.figure.set_figwidth(img.shape[1] / dpi)
    ax.figure.set_figheight(img.shape[0] / dpi)
    ax.figure.canvas.resize(img.shape[1] + 1, img.shape[0] + 1)
    ax.axes.set_position([0, 0, 1, 1])
    ax.axes.set_xlim(-1, img.shape[1])
    ax.axes.set_ylim(img.shape[0], -1)
  plt.grid(False)
  plt.title(title)  
  plt.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!cp -r "/content/drive/My Drive/HackEPS2020/BonArea/data" .

In [ ]:
ima_col = io.imread('data/t485. 08.24.02.jpg')
ima = io.imread('data/t485. 08.24.02.jpg',as_gray=True)
#ima = transform.resize(ima,np.divide(ima.shape,2).astype(int),mode='reflect') 
ima = ima/np.amax(ima)
display_image(ima,size=1)

In [ ]:
def MultiResolution(ima, filter, p1, p2):
  g0 = ima
  if filter == 'gaussian':
    g1 = filters.gaussian(g0,sigma=p1)
    g2 = filters.gaussian(g1,sigma=p2)
  elif filter == 'erosion':
    g1 = morphology.erosion(g0,morphology.disk(p1))
    g2 = morphology.erosion(g1,morphology.disk(p2))
  elif filter == 'dilation':
    g1 = morphology.dilation(g0,morphology.disk(p1))
    g2 = morphology.dilation(g1,morphology.disk(p2))
  elif filter == 'opening':
    g1 = morphology.opening(g0,morphology.disk(p1))
    g2 = morphology.opening(g1,morphology.disk(p2))
  elif filter == 'closing':
    g1 = morphology.closing(g0,morphology.disk(p1))
    g2 = morphology.closing(g1,morphology.disk(p2))
  elif filter == 'alternating':
    tm = morphology.opening(g0,morphology.disk(p1))
    g1 = morphology.closing(tm,morphology.disk(p1))    
    tm = morphology.opening(g1,morphology.disk(p2))
    g2 = morphology.closing(tm,morphology.disk(p2))
  elif filter == 'opening_rec':
    tm = morphology.erosion(g0,morphology.disk(p1))
    g1 = morphology.reconstruction(tm,g0,method='dilation')
    tm = morphology.erosion(g1,morphology.disk(p2))
    g2 = morphology.reconstruction(tm,g1,method='dilation')
  elif filter == 'closing_rec':
    tm = morphology.dilation(g0,morphology.disk(p1))
    g1 = morphology.reconstruction(tm,g0,method='erosion')
    tm = morphology.dilation(g1,morphology.disk(p2))
    g2 = morphology.reconstruction(tm,g1,method='erosion')
  elif filter == 'alternating_rec':
    tm = morphology.erosion(g0,morphology.disk(p1))
    tn = morphology.reconstruction(tm,g0,method='dilation')
    tm = morphology.dilation(tn,morphology.disk(p1))
    g1 = morphology.reconstruction(tm,tn,method='erosion')
    tm = morphology.erosion(g1,morphology.disk(p2))
    tn = morphology.reconstruction(tm,g1,method='dilation')
    tm = morphology.dilation(tn,morphology.disk(p2))
    g2 = morphology.reconstruction(tm,tn,method='erosion')
  else:
    print('Unknown filter')
  l1 = g0 - g1
  l2 = g1 - g2
  return l1, l2, g2
  

In [ ]:
l1, l2, g2 = MultiResolution(ima,'gaussian',9,9)
#display_image(l1,size=1)
#display_image(l2,size=1)
#display_image(g2,size=1)
#display_image(l1+l2+g2,size=1)

In [ ]:
grad_img = cv2.Laplacian(ima,cv2.CV_64F)

In [ ]:
display_image(grad_img, size = 1)

In [ ]:
grad_img.min()

In [ ]:
ret,thresh1 = cv2.threshold(grad_img,0.2,1,cv2.THRESH_BINARY)

In [ ]:
display_image(thresh1, size = 1)

Apply erosions etc...

In [ ]:
se1 = np.ones((17,17))

In [ ]:
ima_dil = nd.grey_dilation(thresh1,     footprint=se1, mode='constant', cval=0)
ima_clo = nd.grey_erosion (ima_dil, footprint=se1, mode='constant', cval=1)

In [ ]:
display_image(ima_clo, size = 1)

Detectar el frigo

In [ ]:
display_image(thresh1, size = 1)

In [ ]:
lines = transform.probabilistic_hough_line(thresh1, threshold=100, line_length=80, line_gap=3)

In [ ]:
for line in lines:
  p0, p1 = line

  plt.plot((p0[0], p1[0]), (-p0[1], -p1[1]))

Color segmentation

In [ ]:
ima_hsv = color.rgb2hsv(ima_col)

In [ ]:
display_image(ima_hsv, size = 1)

In [ ]:
mask = np.zeros([ima_hsv.shape[0], ima_hsv.shape[1]])

In [ ]:
for y in range(mask.shape[0]):
  for x in range(mask.shape[1]):
    if x < 950 and -0.85*x + 880 < y and -1*x + 1350 > y:
      mask[y, x] = 1

In [ ]:
display_image(np.repeat(mask[:, :, np.newaxis], 3, axis=2)*ima_hsv)

In [ ]:
def rgb2hsv(image):
    new_img = color.rgb2hsv(image)
    return new_img


def getMaskedPixels(img, mask):
    inv_mask = mask #mask*-1 + 1
    
    img_reshape = img.reshape(img.shape[0]*img.shape[1], 3)
    mask_reshape = inv_mask.reshape(inv_mask.shape[0]*inv_mask.shape[1])
    
    assert img.shape[0]*img.shape[1] == inv_mask.shape[0]*inv_mask.shape[1]
    
    pixels = np.array([pixel_img for pixel_img, pixel_mask in zip(img_reshape, mask_reshape) if round(pixel_mask) == 1.0])
    
    return pixels

def calculateHist(images, bins = 50, transformation = rgb2hsv):
    values_cr_hist = np.zeros(bins)
    values_cb_hist = np.zeros(bins)
    edges = np.zeros(bins+1)
    for idx, image in enumerate(images):
        new_img = transformation(image)
        masked_image = getMaskedPixels(new_img, mask)
        Cr, Cb = masked_image[:,0]*255, masked_image[:,1]*255
        Cr_hist = np.histogram(Cr, range = (0, 255), bins = bins)
        Cb_hist = np.histogram(Cb, range = (0, 255), bins = bins)
        
        values_cr_hist += Cr_hist[0]
        values_cb_hist += Cb_hist[0]
        edges = Cr_hist[1]
        
    #values_cr_hist[0] = 0
    #values_cb_hist[0] = 0
    
    values_cr_hist /= sum(values_cr_hist)
    values_cb_hist /= sum(values_cb_hist)
    
    
    return edges, values_cr_hist, values_cb_hist


hists = calculateHist([ima_hsv], bins = 100, transformation = rgb2hsv)
edges, values_cr_hist, values_cb_hist = hists

In [ ]:
plt.figure(figsize=(13,5))
plt.subplot(1,2,1)
plt.bar(edges[:-1], values_cb_hist)
plt.title("Saturation")
plt.subplot(1,2,2)
plt.bar(edges[:-1], values_cr_hist)
plt.title("Hue")
plt.show()

In [ ]:
from numba import jit


#Predecir la máscara
def predictMask(image, histograms, transformation = rgb2hsv, th = 0.1):
    shape = image.shape[:-1]
    predict_mask = np.zeros(shape)
    new_img = transformation(image)
    for i in range(shape[0]):
        for j in range(shape[1]):
            predict_mask[i,j] = mask_value(new_img[i,j,:], histograms, th)
            
    return predict_mask

@jit
def mask_value(pixel, hists, th):
    cr = pixel[0]*255
    cb = pixel[1]*255
    edges, values_cr_hist, values_cb_hist = hists
    
    cr_value = 0
    cb_value = 0
    seen_cr = False
    seen_cb = False
    
    for i, edge in enumerate(edges):
        if cr < edge and not seen_cr:
            cr_value = values_cr_hist[i-1]    #Valor respecto el primer histograma
            seen_cr = True
        if cb < edge and not seen_cb:
            cb_value = values_cb_hist[i-1]    #Valor respecto el segundo histograma
            seen_cb = True
            
    final_value = np.array(values_cr_hist).max()/(cr_value+10e-10)
    final_value += np.array(values_cb_hist).max()/(cb_value+10e-10)
    final_value = 2/final_value
    
    #Binarizar
    if final_value > th:
        final_value = 1
    else:
        final_value = 0
    
    return final_value

In [ ]:
mask_pred = predictMask(ima_hsv, hists, transformation = rgb2hsv, th = 0.75)

In [ ]:
display_image(mask_pred, size = 1)

In [ ]:
se1 = np.ones((6,6))
ima_dil = nd.grey_dilation(mask_pred,     footprint=se1, mode='constant', cval=0)
ima_clo = nd.grey_erosion (ima_dil, footprint=se1, mode='constant', cval=1)

In [ ]:
display_image(ima_clo, size = 1)

In [ ]:
markers = measure.label(ima_clo,connectivity=2)

In [ ]:
count_labels = Counter(markers.flatten())
second_most_common_label = count_labels.most_common(2)[1][0]
target_area = (markers == second_most_common_label).astype(float)

In [ ]:
display_image(target_area, size = 1)

In [ ]:
se1 = np.ones((90,90)) 
target_area_dil = nd.grey_dilation(target_area,     footprint=se1, mode='constant', cval=0)
target_area_clo = nd.grey_erosion (target_area_dil, footprint=se1, mode='constant', cval=1)

#se2 = np.ones((120,120)) 
#target_area_dil = nd.grey_dilation(target_area_clo,     footprint=se2, mode='constant', cval=0)

In [ ]:
display_image(target_area_dil, size = 1)

Mix Mask with lines

In [ ]:
shape = target_area_clo.shape

lines_post = []
for i in range(len(lines)):
  p0, p1 = lines[i]

  new_p0 = (p0[0], shape[0]-p0[1])
  new_p1 = (p1[0], shape[0]-p1[1])

  lines_post.append((new_p0, new_p1))

In [ ]:
gradients = []
intercepts = []

def calculate_gradient(p0, p1):
  p0 = np.array(p0)
  p1 = np.array(p1)
  norm = np.linalg.norm(p1-p0)
  x_span = p1[0]-p0[0]

  return np.arccos(x_span/norm)

def calculate_intercept(p0, p1, gradient):
  n = -p0[0]*gradient + p0[1]

  return n


for i, line in enumerate(lines_post):
  p0, p1 = line

  if target_area_clo[p0[1], p0[0]] == 1 or target_area_clo[p1[1], p1[0]] == 1:
    gradient = calculate_gradient(p0, p1)
    gradients.append(gradient)

    intercepts.append(calculate_intercept(p0, p1, gradient))

    plt.plot((p0[0], p1[0]), (p0[1], p1[1]))

In [ ]:
hist = plt.hist(gradients, bins = 20)

In [ ]:
max = hist[1][np.argmax(hist[0])]

filter = [abs(grad - max) < 0.4 for grad in gradients]
gradients_filtered = np.array(gradients)[filter]
intercepts_filtered = np.array(intercepts)[filter]

In [ ]:
hist_2d = []

for g, i in zip(gradients_filtered, intercepts_filtered):
  hist_2d.append((g, i))

In [ ]:
hist = plt.hist2d(gradients_filtered, intercepts_filtered, bins = 20)

In [ ]:
ret,hist_bin = cv2.threshold(hist[0],0.1,1,cv2.THRESH_BINARY)

In [ ]:
display_label(measure.label(hist_bin, connectivity=1))

Remat

In [ ]:
def read_ima(path):
  ima_col = io.imread(path)
  ima = io.imread(path,as_gray=True)
  #ima = transform.resize(ima,np.divide(ima.shape,2).astype(int),mode='reflect') 
  ima = ima/np.amax(ima)

  return ima, ima_col

def get_empty(ima):
  grad_img = cv2.Laplacian(ima,cv2.CV_64F)

  ret,thresh1 = cv2.threshold(grad_img,0.2,1,cv2.THRESH_BINARY)

  se1 = np.ones((17,17))
  ima_dil = nd.grey_dilation(thresh1,     footprint=se1, mode='constant', cval=0)
  ima_clo = nd.grey_erosion (ima_dil, footprint=se1, mode='constant', cval=1)

  return ima_clo

def get_mask(ima_col):
  ima_hsv = color.rgb2hsv(ima_col)
  mask_pred = predictMask(ima_hsv, hists, transformation = rgb2hsv, th = 0.75)
  se1 = np.ones((6,6))
  ima_dil = nd.grey_dilation(mask_pred,     footprint=se1, mode='constant', cval=0)
  ima_clo = nd.grey_erosion (ima_dil, footprint=se1, mode='constant', cval=1)

  markers = measure.label(ima_clo,connectivity=2)
  count_labels = Counter(markers.flatten())
  second_most_common_label = count_labels.most_common(2)[1][0]
  target_area = (markers == second_most_common_label).astype(float)

  se1 = np.ones((90,90)) 
  target_area_dil = nd.grey_dilation(target_area,     footprint=se1, mode='constant', cval=0)
  target_area_clo = nd.grey_erosion (target_area_dil, footprint=se1, mode='constant', cval=1)

  #se2 = np.ones((80,80)) 
  #target_area_dil = nd.grey_dilation(target_area_clo,     footprint=se2, mode='constant', cval=0)

  return target_area_clo

def predict_empty(ima_col, prediction):
  labels_pred = measure.label(prediction, background = 1, connectivity=2)
  labels_pred_counter = Counter(labels_pred.flatten())

  background = labels_pred_counter.most_common(1)[0][0]
  shelf = labels_pred_counter.most_common(2)[1][0]

  #get empty slots
  empty_slots = []
  for value, count in labels_pred_counter.items():
    if value not in (background, shelf):
      if count > 100:
        empty_slots.append(value)
        
  #paint empty slots red and background white  
  labels_pred_red = np.repeat(labels_pred.copy()[:, :, np.newaxis] , 3, axis=2)
  for i in range(labels_pred_red.shape[0]):
    for j in range(labels_pred_red.shape[1]):
      if labels_pred[i,j] in empty_slots:
        labels_pred_red[i,j,:] = [255,0,0]
      else:
        labels_pred_red[i,j,:] = [255,255,255]
        
  #Blurring  
  labels_pred_red_blur = cv2.blur(labels_pred_red,(60,60))

  #Merge original image with mask with red slots 
  ima_col_final = ima_col.copy()
  for i in range(labels_pred_red.shape[0]):
    for j in range(labels_pred_red.shape[1]):
      if labels_pred[i,j] in empty_slots:
        ima_col_final[i,j,:] = labels_pred_red_blur[i,j]

  return ima_col_final

In [ ]:
path = 'data/t485. 08.24.02.jpg'

In [ ]:
ima, ima_col = read_ima(path)
empty = get_empty(ima)
mask_target = get_mask(ima_col)
prediction = predict_empty(ima_col, empty*mask_target)

In [ ]:
display_image(prediction, size = 1)

In [ ]:
def BonArea_algorithm(path):
  ima, ima_col = read_ima(path)
  empty = get_empty(ima)
  mask_target = get_mask(ima_col)
  prediction = predict_empty(ima_col, empty*mask_target)

  return prediction

In [ ]:
import os

input_path = "./data/"
output_path = "./drive/MyDrive/HackEPS2020/BonArea/predictions/"

files = os.listdir(input_path)

for file in files:
  print("Processing file ", file)
  prediction = BonArea_algorithm(input_path + file)
  
  plt.imsave(output_path + "out_" + file, prediction)


In [ ]:
display_image(mask_target * empty)

In [ ]:
prediction = mask_target * empty

In [ ]:
display_image(prediction)

In [ ]:
labels_pred = measure.label(prediction, background = 1, connectivity=2)

In [ ]:
labels_pred_counter = Counter(labels_pred.flatten())

In [ ]:
background = labels_pred_counter.most_common(1)[0][0]
shelf = labels_pred_counter.most_common(2)[1][0]

In [ ]:
empty_slots = []
for value, count in labels_pred_counter.items():
  if value not in (background, shelf):
    if count > 100:
      empty_slots.append(value)

In [ ]:
labels_pred_red = np.repeat(labels_pred.copy()[:, :, np.newaxis] , 3, axis=2)
for i in range(labels_pred_red.shape[0]):
  for j in range(labels_pred_red.shape[1]):
    if labels_pred[i,j] in empty_slots:
      labels_pred_red[i,j,:] = [255,0,0]
    else:
      labels_pred_red[i,j,:] = [255,255,255]

In [ ]:
labels_pred_red_blur = cv2.blur(labels_pred_red,(60,60))

In [ ]:
display_image(labels_pred_red_blur)

In [ ]:
ima_col_final = ima_col.copy()
for i in range(labels_pred_red.shape[0]):
  for j in range(labels_pred_red.shape[1]):
    if labels_pred[i,j] in empty_slots:
      ima_col_final[i,j,:] = labels_pred_red_blur[i,j]

In [ ]:
display_image(ima_col_final, size = 1)

In [ ]:
ima, ima_col = read_ima(path)
################################################
ima_mirror = np.flip(ima, axis = 1)            # Mirrored image
ima_col_mirror = np.flip(ima_col, axis = 1)    # Mirrored image
################################################
empty = get_empty(ima_mirror)
mask_target = get_mask(ima_col_mirror)
prediction = predict_empty(ima_col_mirror, empty*mask_target)

In [ ]:
display_image(ima_col, size = 1)

In [ ]:
display_image(prediction, size = 1)